In [ ]:
%env CUDA_VISIBLE_DEVICES=2
from transformers import  AutoTokenizer
from transformers import LlamaConfig, LlamaForCausalLM
                          

tokenizer = AutoTokenizer.from_pretrained("./tknz/tiny-ko-tokenizer", trust_remote_code=True)
print(len(tokenizer))

context_length = 2048


tokenizer.model_max_length = context_length

config = LlamaConfig(
    # --- 모델 구조 및 크기 관련 핵심 파라미터 ---
    
    hidden_size=480,        # [필수] 모델의 모든 레이어에서 사용되는 벡터의 차원(크기)입니다. 
                            # 임베딩, 어텐션 출력 등 모델의 기본 표현력을 결정하는 가장 중요한 파라미터 중 하나입니다.
                            # 값이 클수록 모델의 용량(capacity)이 커져 더 복잡한 정보를 학습할 수 있지만, 메모리 사용량과 연산량이 크게 증가합니다.
                            # (참고: Llama3-8B는 4096, TinyLlama는 2048을 사용합니다.)

    num_hidden_layers=32,    # [필수] 모델에 쌓을 트랜스포머 블록(레이어)의 총 개수입니다.
                            # 모델의 '깊이'를 결정하며, 깊을수록 더 추상적이고 복잡한 패턴을 학습할 수 있습니다.
                            # hidden_size와 함께 모델의 전체 파라미터 수를 결정하는 핵심 요소입니다.
                            # (참고: Llama3-8B는 32, TinyLlama는 22를 사용합니다.)

    intermediate_size=1920,  # [필수] 각 트랜스포머 블록 내부의 피드포워드 신경망(FFN)의 중간 레이어 크기입니다.
                            # 어텐션 메커니즘이 처리한 정보를 확장했다가 다시 축소하는 역할을 하여 모델의 학습 능력을 높입니다.
                            # 보통 hidden_size의 2.5배 ~ 4배 사이로 설정하며, (1536 / 576 ≈ 2.67배로 적절한 범위)
                            # 최근 모델들은 이 비율을 더 높이는 경향이 있습니다. (예: Llama3 ≈ 3.5배)

    # --- etc, ---

    tie_word_embeddings=True,

    # --- 어텐션 메커니즘 관련 파라미터 ---

    num_attention_heads=6,   # [필수] 멀티 헤드 어텐션(Multi-Head Attention)에서 사용할 '헤드'의 개수입니다.
                            # 하나의 어텐션을 여러 개로 나누어 각각 다른 관점에서 정보의 연관성을 보도록 하는 효과가 있습니다.
                            # `hidden_size`는 반드시 `num_attention_heads`로 나누어떨어져야 합니다. (576 / 9 = 64)
                            # 이 결과값(64)이 각 어텐션 헤드의 차원(head_dim)이 됩니다.
                            # IMPORTANT: self.hidden_size // self.num_attention_heads = head_dim,
                            # head_dim should be one of: [ 256, 192, 128, 96, 80, 64 ]
                            # hidden_size는 이 값으로 나누어져야 합니다. (hidden_size % num_attention_heads == 0)

    num_key_value_heads=2,   # [선택적, 성능향상용] Grouped-Query Attention (GQA)을 위한 파라미터입니다.
                            # 추론 시 속도 향상을 위해 여러 개의 쿼리 헤드(Q)가 하나의 키(K)/밸류(V) 헤드를 공유하도록 합니다.
                            # - `num_key_value_heads` == `num_attention_heads` : 일반적인 Multi-Head Attention (MHA)
                            # - `num_key_value_heads` == 1 : Multi-Query Attention (MQA)
                            # - 1 < `num_key_value_heads` < `num_attention_heads` : Grouped-Query Attention (GQA)
                            # 여기서는 9개의 쿼리 헤드가 2개의 키/밸류 헤드를 공유하며(2개씩 1그룹), 추론 시 메모리 대역폭을 절약하여 속도를 높입니다.
    
    # --- 토크나이저 및 입출력 관련 파라미터 ---
    vocab_size=len(tokenizer),
    max_position_embeddings=context_length,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

print(f"Config: {config}")

# 랜덤 파라미터로 초기화
model = LlamaForCausalLM(config)

# 모델 파라미터 수 확인
model_size = sum(t.numel() for t in model.parameters())
print(f"Model size: {model_size/1000**3:.2f}B parameters")
print(f"Model size: {model_size/1000**2:.2f}M parameters")
print(f"Model size: {model_size/1000:.1f}K parameters")

model.save_pretrained("./tiny-random")
tokenizer.save_pretrained("./tiny-random")

model.push_to_hub("minpeter/tiny-ko-random", private=True)
tokenizer.push_to_hub("minpeter/tiny-ko-random", private=True)

env: CUDA_VISIBLE_DEVICES=2
32018
Config: LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32001,
  "head_dim": 80,
  "hidden_act": "silu",
  "hidden_size": 480,
  "initializer_range": 0.02,
  "intermediate_size": 1920,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 6,
  "num_hidden_layers": 32,
  "num_key_value_heads": 2,
  "pad_token_id": 32003,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 32018
}

Model size: 0.12B parameters
Model size: 123.53M parameters
Model size: 123534.2K parameters


Uploading...:   0%|          | 0.00/494M [00:00<?, ?B/s]

In [4]:
%env CUDA_VISIBLE_DEVICES=6
import transformers
import torch

model_id = "./tiny-random"
model_id = "/data/minpeter/github.com/minpeter/tiny-ko/model/tiny-ko-124m-base"

pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="auto",
)

prompt = """
All:
Content, content.

MENENIUS:
O sir, you are not right: have you not known
The worthiest men have done't?

CORIOLANUS:
""".lstrip()

sampling_output = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    max_new_tokens=2000
)[0]["generated_text"]

print(sampling_output)

env: CUDA_VISIBLE_DEVICES=6


Device set to use cuda:0


All:
Content, content.

MENENIUS:
O sir, you are not right: have you not known
The worthiest men have done't?

CORIOLANUS:
What must I say?
'I Pray, sir'--Plague upon't! I cannot bring
My tongue to such a pace:--'Look, sir, my wounds!
I got them in my country's service, when
Some certain of your brethren roar'd and ran
From the noise of our own drums.'

MENENIUS:
O me, the gods!
You must not speak of that: you must desire them
To think upon you.

CORIOLANUS:
Think upon me! hang 'em!
I would they would forget me, like the virtues
Which our divines lose by 'em.
